# Lab 4: Genetic Algorithms and Evolutionary Computation

## Learning Objectives

By the end of this lab, you will:
- Understand evolutionary computation principles
- Implement genetic algorithms from scratch
- Design encoding schemes for different problems
- Apply selection, crossover, and mutation operators
- Tune GA parameters for optimal performance
- Solve complex optimization problems with GAs

## What are Genetic Algorithms?

**Genetic Algorithms (GAs)** are inspired by biological evolution:
- **Population**: Set of candidate solutions
- **Fitness**: Quality measure for each solution
- **Selection**: Better solutions more likely to reproduce
- **Crossover**: Combine parents to create offspring
- **Mutation**: Random changes for diversity

**Key Idea**: "Survival of the fittest" leads to better solutions over generations!

## Why Genetic Algorithms?

**Advantages**:
- Work on complex, non-differentiable problems
- Explore multiple solutions in parallel
- Handle discrete and continuous variables
- Escape local optima naturally
- No gradient information needed

**Trade-offs**:
- More evaluations than gradient methods
- Many parameters to tune
- No guarantee of optimality

## Real-World Applications

- 🧬 **Drug Design**: Molecular optimization
- 🎨 **Design**: Architecture, circuit layout
- 🤖 **Robotics**: Controller evolution
- 📊 **Finance**: Portfolio optimization
- 🎮 **Game AI**: Strategy evolution
- 🧠 **Neural Architecture Search**: AutoML


In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Callable, Optional
import random
from copy import deepcopy

# Set random seed
np.random.seed(42)
random.seed(42)

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

## Part 1: Basic Genetic Algorithm

### GA Components

1. **Representation**: How to encode solutions (binary, real-valued, permutation)
2. **Fitness Function**: How to evaluate solution quality
3. **Selection**: How to choose parents (tournament, roulette wheel)
4. **Crossover**: How to combine parents (one-point, two-point, uniform)
5. **Mutation**: How to introduce variation (bit flip, gaussian)

### Basic GA Algorithm

```
1. Initialize population randomly
2. Evaluate fitness of each individual
3. While not converged:
   a. Select parents based on fitness
   b. Create offspring via crossover
   c. Mutate offspring
   d. Evaluate offspring fitness
   e. Replace population (generational or steady-state)
4. Return best solution
```


In [ ]:
class GeneticAlgorithm:
    """Basic Genetic Algorithm implementation."""
    
    def __init__(self, 
                 fitness_fn: Callable,
                 chromosome_length: int,
                 population_size: int = 100,
                 mutation_rate: float = 0.01,
                 crossover_rate: float = 0.8,
                 maximize: bool = True):
        """
        Initialize Genetic Algorithm.
        
        Args:
            fitness_fn: Function to evaluate fitness
            chromosome_length: Length of binary chromosome
            population_size: Number of individuals
            mutation_rate: Probability of bit flip
            crossover_rate: Probability of crossover
            maximize: If True, maximize fitness; else minimize
        """
        self.fitness_fn = fitness_fn
        self.chromosome_length = chromosome_length
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.maximize = maximize
        
        self.population = []
        self.fitness_history = []
    
    def initialize_population(self):
        """Create random initial population."""
        self.population = [
            np.random.randint(0, 2, self.chromosome_length)
            for _ in range(self.population_size)
        ]
    
    def evaluate_population(self) -> List[float]:
        """Evaluate fitness of all individuals."""
        return [self.fitness_fn(ind) for ind in self.population]
    
    def tournament_selection(self, fitness_values: List[float], 
                           tournament_size: int = 3) -> np.ndarray:
        """Select individual using tournament selection."""
        # Randomly select tournament_size individuals
        tournament_indices = random.sample(range(len(self.population)), 
                                          tournament_size)
        tournament_fitness = [fitness_values[i] for i in tournament_indices]
        
        # Select best (or worst if minimizing)
        if self.maximize:
            winner_idx = tournament_indices[np.argmax(tournament_fitness)]
        else:
            winner_idx = tournament_indices[np.argmin(tournament_fitness)]
        
        return self.population[winner_idx].copy()
    
    def one_point_crossover(self, parent1: np.ndarray, 
                           parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        """One-point crossover."""
        if random.random() > self.crossover_rate:
            return parent1.copy(), parent2.copy()
        
        # Choose crossover point
        point = random.randint(1, self.chromosome_length - 1)
        
        # Create offspring
        offspring1 = np.concatenate([parent1[:point], parent2[point:]])
        offspring2 = np.concatenate([parent2[:point], parent1[point:]])
        
        return offspring1, offspring2
    
    def mutate(self, individual: np.ndarray) -> np.ndarray:
        """Bit-flip mutation."""
        mutated = individual.copy()
        for i in range(len(mutated)):
            if random.random() < self.mutation_rate:
                mutated[i] = 1 - mutated[i]  # Flip bit
        return mutated
    
    def evolve(self, generations: int = 100) -> Tuple[np.ndarray, float]:
        """Run genetic algorithm for specified generations."""
        self.initialize_population()
        self.fitness_history = []
        
        for generation in range(generations):
            # Evaluate fitness
            fitness_values = self.evaluate_population()
            
            # Track best fitness
            if self.maximize:
                best_fitness = max(fitness_values)
                best_idx = np.argmax(fitness_values)
            else:
                best_fitness = min(fitness_values)
                best_idx = np.argmin(fitness_values)
            
            self.fitness_history.append(best_fitness)
            
            # Create new population
            new_population = []
            
            # Elitism: keep best individual
            new_population.append(self.population[best_idx].copy())
            
            # Generate rest of population
            while len(new_population) < self.population_size:
                # Selection
                parent1 = self.tournament_selection(fitness_values)
                parent2 = self.tournament_selection(fitness_values)
                
                # Crossover
                offspring1, offspring2 = self.one_point_crossover(parent1, parent2)
                
                # Mutation
                offspring1 = self.mutate(offspring1)
                offspring2 = self.mutate(offspring2)
                
                new_population.extend([offspring1, offspring2])
            
            # Trim to population size
            self.population = new_population[:self.population_size]
        
        # Return best solution
        final_fitness = self.evaluate_population()
        if self.maximize:
            best_idx = np.argmax(final_fitness)
        else:
            best_idx = np.argmin(final_fitness)
        
        return self.population[best_idx], final_fitness[best_idx]


# Example: Maximize number of ones (OneMax problem)
def onemax_fitness(chromosome: np.ndarray) -> float:
    """Fitness = number of 1s in chromosome."""
    return np.sum(chromosome)

print("OneMax Problem: Maximize number of 1s")
print("=" * 60)

ga = GeneticAlgorithm(
    fitness_fn=onemax_fitness,
    chromosome_length=50,
    population_size=100,
    mutation_rate=0.01,
    crossover_rate=0.8,
    maximize=True
)

best_solution, best_fitness = ga.evolve(generations=100)

print(f"Best solution: {''.join(map(str, best_solution[:20]))}... (showing first 20 bits)")
print(f"Best fitness: {best_fitness}/{len(best_solution)}")
print(f"Percentage of 1s: {best_fitness/len(best_solution)*100:.1f}%")

# Plot convergence
plt.figure(figsize=(12, 6))
plt.plot(ga.fitness_history, linewidth=2, color='blue')
plt.axhline(y=len(best_solution), color='r', linestyle='--', 
           label='Optimal', linewidth=2)
plt.xlabel('Generation', fontweight='bold', fontsize=12)
plt.ylabel('Best Fitness', fontweight='bold', fontsize=12)
plt.title('GA Convergence on OneMax Problem', fontweight='bold', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Part 2: Real-Valued Genetic Algorithm

For continuous optimization, we use real-valued chromosomes instead of binary.

### Operators for Real-Valued GAs

**Crossover**:
- Blend crossover (BLX-α)
- Simulated binary crossover (SBX)

**Mutation**:
- Gaussian mutation
- Polynomial mutation


In [ ]:
class RealValuedGA:
    """Genetic Algorithm for continuous optimization."""
    
    def __init__(self,
                 fitness_fn: Callable,
                 bounds: List[Tuple[float, float]],
                 population_size: int = 100,
                 mutation_rate: float = 0.1,
                 crossover_rate: float = 0.8,
                 maximize: bool = False):
        """
        Initialize Real-Valued GA.
        
        Args:
            fitness_fn: Function to evaluate fitness
            bounds: List of (min, max) for each dimension
            population_size: Number of individuals
            mutation_rate: Probability of mutation
            crossover_rate: Probability of crossover
            maximize: If True, maximize; else minimize
        """
        self.fitness_fn = fitness_fn
        self.bounds = bounds
        self.n_dims = len(bounds)
        self.population_size = population_size
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.maximize = maximize
        
        self.population = []
        self.best_history = []
        self.mean_history = []
    
    def initialize_population(self):
        """Create random initial population within bounds."""
        self.population = []
        for _ in range(self.population_size):
            individual = np.array([
                np.random.uniform(low, high)
                for low, high in self.bounds
            ])
            self.population.append(individual)
    
    def evaluate_population(self) -> List[float]:
        """Evaluate fitness of all individuals."""
        return [self.fitness_fn(ind) for ind in self.population]
    
    def tournament_selection(self, fitness_values: List[float]) -> np.ndarray:
        """Tournament selection."""
        tournament_size = 3
        tournament_indices = random.sample(range(len(self.population)), 
                                          tournament_size)
        tournament_fitness = [fitness_values[i] for i in tournament_indices]
        
        if self.maximize:
            winner_idx = tournament_indices[np.argmax(tournament_fitness)]
        else:
            winner_idx = tournament_indices[np.argmin(tournament_fitness)]
        
        return self.population[winner_idx].copy()
    
    def blend_crossover(self, parent1: np.ndarray, 
                       parent2: np.ndarray, alpha: float = 0.5) -> np.ndarray:
        """Blend crossover (BLX-α)."""
        if random.random() > self.crossover_rate:
            return parent1.copy()
        
        offspring = np.zeros(self.n_dims)
        for i in range(self.n_dims):
            min_val = min(parent1[i], parent2[i])
            max_val = max(parent1[i], parent2[i])
            range_val = max_val - min_val
            
            # Sample from [min - α*range, max + α*range]
            offspring[i] = np.random.uniform(
                min_val - alpha * range_val,
                max_val + alpha * range_val
            )
            
            # Clip to bounds
            offspring[i] = np.clip(offspring[i], 
                                  self.bounds[i][0], 
                                  self.bounds[i][1])
        
        return offspring
    
    def gaussian_mutation(self, individual: np.ndarray, 
                         sigma: float = 0.1) -> np.ndarray:
        """Gaussian mutation."""
        mutated = individual.copy()
        for i in range(self.n_dims):
            if random.random() < self.mutation_rate:
                # Add gaussian noise
                range_val = self.bounds[i][1] - self.bounds[i][0]
                mutated[i] += np.random.normal(0, sigma * range_val)
                
                # Clip to bounds
                mutated[i] = np.clip(mutated[i], 
                                    self.bounds[i][0], 
                                    self.bounds[i][1])
        return mutated
    
    def evolve(self, generations: int = 100) -> Tuple[np.ndarray, float]:
        """Run genetic algorithm."""
        self.initialize_population()
        self.best_history = []
        self.mean_history = []
        
        for generation in range(generations):
            # Evaluate
            fitness_values = self.evaluate_population()
            
            # Track statistics
            if self.maximize:
                best_fitness = max(fitness_values)
                best_idx = np.argmax(fitness_values)
            else:
                best_fitness = min(fitness_values)
                best_idx = np.argmin(fitness_values)
            
            self.best_history.append(best_fitness)
            self.mean_history.append(np.mean(fitness_values))
            
            # Create new population
            new_population = []
            
            # Elitism
            new_population.append(self.population[best_idx].copy())
            
            # Generate offspring
            while len(new_population) < self.population_size:
                parent1 = self.tournament_selection(fitness_values)
                parent2 = self.tournament_selection(fitness_values)
                
                offspring = self.blend_crossover(parent1, parent2)
                offspring = self.gaussian_mutation(offspring)
                
                new_population.append(offspring)
            
            self.population = new_population[:self.population_size]
        
        # Return best
        final_fitness = self.evaluate_population()
        if self.maximize:
            best_idx = np.argmax(final_fitness)
        else:
            best_idx = np.argmin(final_fitness)
        
        return self.population[best_idx], final_fitness[best_idx]


# Test on benchmark functions
def sphere(x):
    """Sphere function: simple convex."""
    return np.sum(x**2)

def rastrigin(x):
    """Rastrigin: highly multimodal."""
    A = 10
    return A * len(x) + np.sum(x**2 - A * np.cos(2 * np.pi * x))

print("\nReal-Valued GA on Benchmark Functions")
print("=" * 60)

# Test functions
test_functions = [
    ("Sphere", sphere, 2),
    ("Rastrigin", rastrigin, 2)
]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, (name, func, dims) in zip(axes, test_functions):
    bounds = [(-5, 5)] * dims
    
    ga = RealValuedGA(
        fitness_fn=func,
        bounds=bounds,
        population_size=50,
        mutation_rate=0.1,
        crossover_rate=0.8,
        maximize=False
    )
    
    best_sol, best_fit = ga.evolve(generations=200)
    
    print(f"{name}:")
    print(f"  Best solution: {best_sol}")
    print(f"  Best fitness: {best_fit:.6f}")
    print()
    
    # Plot convergence
    ax.plot(ga.best_history, label='Best', linewidth=2)
    ax.plot(ga.mean_history, label='Mean', linewidth=2, alpha=0.7)
    ax.set_xlabel('Generation', fontweight='bold')
    ax.set_ylabel('Fitness', fontweight='bold')
    ax.set_title(f'{name} Function', fontweight='bold')
    ax.set_yscale('log')
    ax.legend()
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Part 3: TSP with Genetic Algorithm

For permutation problems like TSP, we need specialized operators:

**Crossover**:
- Order Crossover (OX)
- Partially Mapped Crossover (PMX)

**Mutation**:
- Swap mutation
- Inversion mutation


In [ ]:
class TSP_GA:
    """Genetic Algorithm for Traveling Salesman Problem."""
    
    def __init__(self, cities: np.ndarray, population_size: int = 100):
        """
        Initialize TSP GA.
        
        Args:
            cities: Array of city coordinates (n_cities, 2)
            population_size: Number of tours in population
        """
        self.cities = cities
        self.n_cities = len(cities)
        self.population_size = population_size
        
        # Precompute distances
        self.distances = np.zeros((self.n_cities, self.n_cities))
        for i in range(self.n_cities):
            for j in range(self.n_cities):
                self.distances[i, j] = np.linalg.norm(cities[i] - cities[j])
        
        self.population = []
        self.best_history = []
    
    def tour_length(self, tour: List[int]) -> float:
        """Calculate total tour length."""
        length = 0
        for i in range(len(tour)):
            length += self.distances[tour[i], tour[(i + 1) % len(tour)]]
        return length
    
    def initialize_population(self):
        """Create random initial population."""
        self.population = []
        for _ in range(self.population_size):
            tour = list(range(self.n_cities))
            random.shuffle(tour)
            self.population.append(tour)
    
    def tournament_selection(self, fitness_values: List[float]) -> List[int]:
        """Tournament selection."""
        tournament_size = 5
        tournament_indices = random.sample(range(len(self.population)), 
                                          tournament_size)
        tournament_fitness = [fitness_values[i] for i in tournament_indices]
        winner_idx = tournament_indices[np.argmin(tournament_fitness)]  # Minimize distance
        return self.population[winner_idx][:]
    
    def order_crossover(self, parent1: List[int], 
                       parent2: List[int]) -> List[int]:
        """Order Crossover (OX)."""
        size = len(parent1)
        
        # Choose two random crossover points
        start, end = sorted(random.sample(range(size), 2))
        
        # Copy segment from parent1
        offspring = [-1] * size
        offspring[start:end] = parent1[start:end]
        
        # Fill remaining from parent2
        current_pos = end
        for city in parent2[end:] + parent2[:end]:
            if city not in offspring:
                offspring[current_pos % size] = city
                current_pos += 1
        
        return offspring
    
    def swap_mutation(self, tour: List[int], rate: float = 0.1) -> List[int]:
        """Swap mutation."""
        mutated = tour[:]
        if random.random() < rate:
            i, j = random.sample(range(len(tour)), 2)
            mutated[i], mutated[j] = mutated[j], mutated[i]
        return mutated
    
    def evolve(self, generations: int = 500) -> Tuple[List[int], float]:
        """Run genetic algorithm."""
        self.initialize_population()
        self.best_history = []
        
        for generation in range(generations):
            # Evaluate
            fitness_values = [self.tour_length(tour) for tour in self.population]
            
            # Track best
            best_idx = np.argmin(fitness_values)
            best_fitness = fitness_values[best_idx]
            self.best_history.append(best_fitness)
            
            # Create new population
            new_population = []
            
            # Elitism
            new_population.append(self.population[best_idx][:])
            
            # Generate offspring
            while len(new_population) < self.population_size:
                parent1 = self.tournament_selection(fitness_values)
                parent2 = self.tournament_selection(fitness_values)
                
                offspring = self.order_crossover(parent1, parent2)
                offspring = self.swap_mutation(offspring, rate=0.2)
                
                new_population.append(offspring)
            
            self.population = new_population[:self.population_size]
        
        # Return best
        final_fitness = [self.tour_length(tour) for tour in self.population]
        best_idx = np.argmin(final_fitness)
        return self.population[best_idx], final_fitness[best_idx]


# Test on TSP
print("Genetic Algorithm for TSP")
print("=" * 60)

# Generate random cities
np.random.seed(42)
n_cities = 20
cities = np.random.rand(n_cities, 2) * 100

tsp_ga = TSP_GA(cities, population_size=100)

print(f"Solving TSP with {n_cities} cities...")
best_tour, best_length = tsp_ga.evolve(generations=500)

print(f"Best tour length: {best_length:.2f}")
print()

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot tour
tour_coords = np.array([cities[i] for i in best_tour + [best_tour[0]]])
ax1.plot(tour_coords[:, 0], tour_coords[:, 1], 'b-', linewidth=2, alpha=0.6)
ax1.scatter(cities[:, 0], cities[:, 1], s=200, c='red', 
           zorder=5, edgecolors='black', linewidths=2)
ax1.set_xlabel('X', fontweight='bold')
ax1.set_ylabel('Y', fontweight='bold')
ax1.set_title(f'Best Tour (Length: {best_length:.2f})', 
             fontweight='bold', fontsize=13)
ax1.grid(alpha=0.3)
ax1.axis('equal')

# Plot convergence
ax2.plot(tsp_ga.best_history, linewidth=2, color='blue')
ax2.set_xlabel('Generation', fontweight='bold')
ax2.set_ylabel('Best Tour Length', fontweight='bold')
ax2.set_title('GA Convergence on TSP', fontweight='bold', fontsize=13)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Part 4: Feature Selection with GA

### Application: Feature Selection for ML

**Problem**: Select subset of features that maximizes model performance.

**Encoding**: Binary chromosome where 1 = include feature, 0 = exclude


In [ ]:
# Feature selection example (synthetic data)
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

print("Feature Selection with Genetic Algorithm")
print("=" * 60)

# Generate synthetic classification dataset
X, y = make_classification(
    n_samples=200,
    n_features=20,
    n_informative=10,
    n_redundant=5,
    n_clusters_per_class=2,
    random_state=42
)

print(f"Dataset: {X.shape[0]} samples, {X.shape[1]} features")
print()

def feature_selection_fitness(chromosome: np.ndarray) -> float:
    """
    Fitness = cross-validation accuracy with selected features.
    Penalty for too many features.
    """
    # Select features
    selected_features = np.where(chromosome == 1)[0]
    
    if len(selected_features) == 0:
        return 0.0  # No features selected
    
    # Train and evaluate model
    X_selected = X[:, selected_features]
    clf = DecisionTreeClassifier(max_depth=5, random_state=42)
    scores = cross_val_score(clf, X_selected, y, cv=3, scoring='accuracy')
    accuracy = scores.mean()
    
    # Penalty for using too many features (encourage parsimony)
    n_features_penalty = len(selected_features) / len(chromosome) * 0.1
    
    return accuracy - n_features_penalty

# Run GA for feature selection
ga_fs = GeneticAlgorithm(
    fitness_fn=feature_selection_fitness,
    chromosome_length=X.shape[1],
    population_size=50,
    mutation_rate=0.05,
    crossover_rate=0.8,
    maximize=True
)

best_features, best_fitness = ga_fs.evolve(generations=50)

selected = np.where(best_features == 1)[0]
print(f"Best fitness: {best_fitness:.4f}")
print(f"Selected {len(selected)} features: {selected.tolist()}")
print()

# Compare with using all features
clf_all = DecisionTreeClassifier(max_depth=5, random_state=42)
scores_all = cross_val_score(clf_all, X, y, cv=3, scoring='accuracy')
accuracy_all = scores_all.mean()

clf_selected = DecisionTreeClassifier(max_depth=5, random_state=42)
scores_selected = cross_val_score(clf_selected, X[:, selected], y, cv=3, scoring='accuracy')
accuracy_selected = scores_selected.mean()

print("Comparison:")
print(f"  All features ({X.shape[1]}): {accuracy_all:.4f}")
print(f"  Selected features ({len(selected)}): {accuracy_selected:.4f}")
print()
print("GA found a compact feature set with similar performance!")

# Plot convergence
plt.figure(figsize=(12, 6))
plt.plot(ga_fs.fitness_history, linewidth=2, color='green')
plt.axhline(y=accuracy_all, color='r', linestyle='--', 
           label=f'All features: {accuracy_all:.3f}', linewidth=2)
plt.xlabel('Generation', fontweight='bold', fontsize=12)
plt.ylabel('Fitness', fontweight='bold', fontsize=12)
plt.title('Feature Selection with GA', fontweight='bold', fontsize=14)
plt.legend(fontsize=11)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## Part 5: Parameter Tuning

### Impact of GA Parameters

Key parameters to tune:
- **Population size**: Larger = more diversity, slower
- **Mutation rate**: Higher = more exploration
- **Crossover rate**: Usually 0.6-0.9
- **Selection pressure**: Tournament size


In [ ]:
print("Parameter Sensitivity Analysis")
print("=" * 60)

# Test function
def rastrigin_2d(x):
    A = 10
    return A * 2 + np.sum(x**2 - A * np.cos(2 * np.pi * x))

# Test different mutation rates
mutation_rates = [0.01, 0.05, 0.1, 0.2]
results = []

for mr in mutation_rates:
    ga = RealValuedGA(
        fitness_fn=rastrigin_2d,
        bounds=[(-5, 5), (-5, 5)],
        population_size=50,
        mutation_rate=mr,
        crossover_rate=0.8,
        maximize=False
    )
    
    best_sol, best_fit = ga.evolve(generations=100)
    results.append((mr, ga.best_history))
    print(f"Mutation rate {mr:.2f}: Final fitness = {best_fit:.4f}")

# Plot comparison
plt.figure(figsize=(12, 6))
for mr, history in results:
    plt.plot(history, label=f'Mutation rate = {mr}', linewidth=2)

plt.xlabel('Generation', fontweight='bold', fontsize=12)
plt.ylabel('Best Fitness', fontweight='bold', fontsize=12)
plt.title('Impact of Mutation Rate on Convergence', fontweight='bold', fontsize=14)
plt.legend(fontsize=11)
plt.yscale('log')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print("\nObservations:")
print("- Low mutation: Fast initial convergence, may get stuck")
print("- High mutation: Slower but better exploration")
print("- Sweet spot typically around 0.05-0.1")

## Exercises

### Exercise 1: Custom Fitness Function
Create a custom optimization problem and solve it with GA.
Compare results with different parameter settings.

In [ ]:
# TODO: Design and solve custom problem
# Your code here
pass

### Exercise 2: Knapsack with GA
Solve the 0-1 knapsack problem from Lab 1 using genetic algorithm.
Compare with greedy solution.

In [ ]:
# TODO: Implement knapsack with GA
# Your code here
pass

### Exercise 3: Multi-Objective Optimization
Implement NSGA-II for multi-objective optimization.
Solve a problem with two conflicting objectives.

In [ ]:
# TODO: Implement multi-objective GA
# Your code here
pass

## Summary

### Key Takeaways

1. **Evolution-inspired** - Population, selection, variation
2. **Encoding matters** - Binary, real-valued, permutation
3. **Operators** - Crossover and mutation for exploration
4. **Selection pressure** - Balance exploitation vs exploration
5. **Parameter tuning** - Critical for performance
6. **Applications** - TSP, feature selection, design optimization

### When to Use GAs

**Good fit**:
- Complex, non-differentiable objectives
- Large search spaces
- Multiple local optima
- Discrete or mixed variables
- Black-box optimization

**Not ideal**:
- Small problems (use exact methods)
- Smooth, convex problems (use gradient methods)
- Need guaranteed optimality
- Very tight time constraints

### GA Variants

- **NSGA-II**: Multi-objective optimization
- **CMA-ES**: Covariance matrix adaptation
- **Differential Evolution**: Alternative operators
- **Genetic Programming**: Evolve programs/trees
- **Neuroevolution**: Evolve neural networks

### Week 4 Complete!

You've now mastered:
1. **Optimization fundamentals** (Lab 1)
2. **Local search** (Lab 2)
3. **Constraint satisfaction** (Lab 3)
4. **Evolutionary algorithms** (Lab 4)

Ready for **Week 5: Machine Learning**! 🚀
